# Deep Learning Toolkit for Splunk - Notebook for PyTorch

## Logistic Regression in PyTorch
This notebook contains an example for a simple logistic regression in PyTorch.<br>By default every time you save this notebook the cells are exported into a python module which is then used for executing your custom model invoked by Splunk MLTK Container App. 

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [ ]:
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import datetime
import numpy as np
import scipy as sp
import pandas as pd
import torch
# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing purposes
print("numpy version: " + np.__version__)
print("scipy version: " + sp.__version__)
print("pandas version: " + pd.__version__)
print("PyTorch: " + torch.__version__)
if torch.cuda.is_available():
    print(f"There are {torch.cuda.device_count()} CUDA devices available")
    for i in range(0,torch.cuda.device_count()):
        print(f"Device {i:0}: {torch.cuda.get_device_name(i)} ")
else:
    print("No GPU found")

## Stage 1 - get a data sample from Splunk
In Splunk run a search to pipe a prepared sample dataset into this environment.

| inputlookup iris.csv <br>
| fit MLTKContainer mode=stage algo=pytorch_nn epochs=10 species from petal_length petal_width sepal_length sepal_width into app:PyTorch_iris_model_nn

After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword ("PyTorch_iris_model" in the example above) or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [ ]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing purposes
df, param = stage("PyTorch_iris_model_nn")
#print(param)
print(df.describe)

## Stage 2 - create and initialize a model

In [ ]:
def init(df,param):
    X = df[param['feature_variables']]
    Y = df[param['target_variables']]
    input_size = int(X.shape[1])
    num_classes = len(np.unique(Y.to_numpy()))
    learning_rate = 0.001
    mapping = { key: value for value,key in enumerate(np.unique(Y.to_numpy().reshape(-1))) }
    print("FIT build neural network model with input shape " + str(X.shape))
    print("FIT build model with target classes " + str(num_classes))
    model = {
        "input_size": input_size,
        "num_classes": num_classes,
        "learning_rate": learning_rate,
        "mapping": mapping,
        "num_epochs": 10000,
        "batch_size": 100,
        "hidden_layers" : 10,
    }
    device = None
    if torch.cuda.is_available():
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')
    model['device'] = device
    
    if 'options' in param:
        if 'params' in param['options']:
            if 'epochs' in param['options']['params']:
                model['num_epochs'] = int(param['options']['params']['epochs'])
            if 'batch_size' in param['options']['params']:
                model['batch_size'] = int(param['options']['params']['batch_size'])
            if 'hidden_layers' in param['options']['params']:
                model['hidden_layers'] = int(param['options']['params']['hidden_layers'])

    # Simple neural network model
    model['model'] = torch.nn.Sequential(
        torch.nn.Linear(model['input_size'], model['hidden_layers']),
        torch.nn.ReLU(),
        torch.nn.Linear(model['hidden_layers'], model['num_classes']),
    ).to(model['device'])

    # Define loss and optimizer
    model['criterion'] = torch.nn.CrossEntropyLoss()  
    model['optimizer'] = torch.optim.SGD(model['model'].parameters(), lr=learning_rate)      
    return model

In [ ]:
model = init(df,param)
print(model)

## Stage 3 - fit the model

In [ ]:
def fit(model,df,param):
    returns = {}
    X = df[param['feature_variables']].astype('float32').to_numpy()
    Y = df[param['target_variables']].to_numpy().reshape(-1)
    mapping = { key: value for value,key in enumerate(np.unique(Y)) }
    Y = df[param['target_variables']].replace( {param['target_variables'][0]:mapping } ).to_numpy().reshape(-1)
    if 'options' in param:
        if 'params' in param['options']:
            if 'epochs' in param['options']['params']:
                model['num_epochs'] = int(param['options']['params']['epochs'])
            if 'batch_size' in param['options']['params']:
                model['batch_size'] = int(param['options']['params']['batch_size'])
    print(model['num_epochs'])

    inputs = torch.from_numpy(X).to(model['device'])
    targets = torch.from_numpy(Y).to(model['device'])

    for epoch in range(model['num_epochs']):
        outputs = model['model'](inputs)
        loss = model['criterion'](outputs, targets)
        model['optimizer'].zero_grad()
        loss.backward()
        model['optimizer'].step()
        if (epoch+1) % (model['num_epochs']/100) == 0:
            print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, model['num_epochs'], loss.item()))                
    # memorize parameters
    returns['model_epochs'] = model['num_epochs']
    returns['model_batch_size'] = model['batch_size']
    returns['model_loss_acc'] = loss.item()
    return returns

In [ ]:
returns = fit(model,df,param)
print(returns['model_loss_acc'])

## Stage 4 - apply the model

In [ ]:
def apply(model,df,param):
    X = df[param['feature_variables']].astype('float32').to_numpy()
    classes = {v: k for k, v in model['mapping'].items()}
    with torch.no_grad():
        input = torch.from_numpy(X).to(model['device'])
        output = model['model'](input)
        y_hat = output.data
        _, predicted = torch.max(output.data, 1)
        predicted = predicted.cpu()
        prediction = [classes[key] for key in predicted.numpy()]
    return prediction

In [ ]:
y_hat = apply(model,df,param)
y_hat

## Stage 5 - save the model

In [ ]:
# save model to name in expected convention "<algo_name>_<model_name>.h5"
def save(model,name):
    torch.save(model, MODEL_DIRECTORY + name + ".pt")
    return model

## Stage 6 - load the model

In [ ]:
# load model from name in expected convention "<algo_name>_<model_name>.h5"
def load(name):
    model = torch.load(MODEL_DIRECTORY + name + ".pt")
    return model

## Stage 7 - provide a summary of the model

In [ ]:
# return model summary
def summary(model=None):
    returns = {"version": {"pytorch": torch.__version__} }
    if model is not None:
        if 'model' in model:
            returns["summary"] = str(model)
    return returns

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code